In [1]:
# import packages
from sma import utils, run_snscrape, sentiment
import pandas as pd
import re
from tqdm import tqdm
import os
from os import listdir

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fergu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk

### Pull intial data

In [8]:
# define the terms and date range for which we wish to pull tweets

'''
after significant testing i found that pulling large volumes of tweets in a single go can lead to excessive run times.
this appears to result from the fact that the time taken to concert the response object - returned by snscrape - to a 
dataframe increases enormously as the number of tweets returned increases. To avoid this, i suggest slicing your date
range to smaller increments and iteratively pulling tweets for each
'''
#terms = ['natwest', 'barclays', 'santander', 'lloyds banking group']
#timeframe = ['2021-01-01', '2021-12-31']

term = 'natwest'
timeframes = [
    ['2021-01-01', '2021-02-28'],
    ['2021-03-01', '2021-04-30'],
    ['2021-05-01', '2021-06-30'],
    ['2021-07-01', '2021-08-31'],
    ['2021-09-01', '2021-10-31'],
    ['2021-11-01', '2021-12-31']
]


In [ ]:
# pull tweets for each timeframe in term
for timeframe in tqdm(timeframes):
    return_df = run_snscrape.scrape_twitter(term, date_range=timeframe, lang='en', limit=None)
    
    # save the output
    output_path = 'data/raw_{}_{}.pickle'.format(term, timeframe[0])
    return_df.to_pickle(output_path)

In [ ]:
# consolidate files to create complete dataset
consolidated_df = pd.DataFrame()
for file in listdir('data'):
    if 'raw_{}'.format(term) in file:
        temp = pd.read_pickle('data/{}'.format(file))
        consolidated_df = pd.concat([consolidated_df, temp], axis=0)

#remove duplicate tweets
consolidated_df = consolidated_df.drop_duplicates(subset='content', ignore_index=True)

print(f'total tweet count: {len(consolidated_df)}')

# save a file format that we can view to manually explore the data
consolidated_df.to_csv(f'data/master_{term}.csv')
# save a version in a format that enables more efficient storage and loading
consolidated_df.to_pickle(f'data/master_{term}.pickle')

### Tag sentiment

In [9]:
master_df = pd.read_pickle(f'data/master_{term}.pickle')

# we will use Vader for sentiment, and return the full polarity output
tqdm.pandas()
new_cols = ['sent_positive', 'sent_negative', 'sent_neutral', 'sent_compound', 'sentiment']
master_df[new_cols] = master_df.progress_apply(lambda x: sentiment.vader_sentiment(x['content'], return_polarities=True), axis=1, result_type='expand')

# save file
master_df.to_pickle(f'data/sentiment_{term}.pickle')
master_df.to_csv(f'data/sentiment_{term}.csv')

100%|██████████| 76145/76145 [05:55<00:00, 214.01it/s]


### Unsupervised clustering